In [1]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.callbacks import EarlyStopping
import matplotlib.pyplot as plt
import cv2
import numpy as np

In [2]:
(train_images, train_labels), (test_images, test_labels) = datasets.cifar10.load_data()

170508288/170498071 [==============================] - 2s 0us/step


In [3]:
print("Train Images Shape : " + str(train_images.shape))
print("Test Images Shape : " + str(test_images.shape))
# Normalize pixel values to be between 0 and 1
train_images, test_images = train_images / 255.0, test_images / 255.0

Train Images Shape : (50000, 32, 32, 3)
Test Images Shape : (10000, 32, 32, 3)


In [4]:
# The classes in CIFAR-10 dataset
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']

In [5]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [7]:
model = models.Sequential()
model.add(data_augmentation)
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
# model.summary()

In [9]:
model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dropout(0.25))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dropout(0.10))
model.add(layers.Dense(10, activation='softmax'))
# model.summary()

In [10]:
earlystop_callback = EarlyStopping(
  monitor='val_loss', min_delta=0, patience=3)

In [11]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])

history = model.fit(train_images, train_labels, epochs=15, verbose=1,
                    validation_data=(test_images, test_labels), shuffle=True, callbacks=[earlystop_callback])
model.summary()
test_loss, test_accuracy = model.evaluate(test_images, test_labels)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100 ))


Epoch 1/15
1563/1563 [==============================] - 45s 11ms/step - loss: 2.0638 - accuracy: 0.1995 - val_loss: 1.9093 - val_accuracy: 0.2564
Epoch 2/15
1563/1563 [==============================] - 16s 10ms/step - loss: 1.8991 - accuracy: 0.2619 - val_loss: 1.8285 - val_accuracy: 0.2907
Epoch 3/15
1563/1563 [==============================] - 16s 10ms/step - loss: 1.7900 - accuracy: 0.3088 - val_loss: 1.6645 - val_accuracy: 0.3551
Epoch 4/15
1563/1563 [==============================] - 16s 10ms/step - loss: 1.7045 - accuracy: 0.3547 - val_loss: 1.6435 - val_accuracy: 0.3794
Epoch 5/15
1563/1563 [==============================] - 16s 10ms/step - loss: 1.6462 - accuracy: 0.3840 - val_loss: 1.5447 - val_accuracy: 0.4096
Epoch 6/15
1563/1563 [==============================] - 16s 11ms/step - loss: 1.6037 - accuracy: 0.4030 - val_loss: 1.4972 - val_accuracy: 0.4392
Epoch 7/15
1563/1563 [==============================] - 17s 11ms/step - loss: 1.5671 - accuracy: 0.4229 - val_loss: 1.4705 -

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0.5, 1])
plt.legend(loc='lower right')

test_loss, test_acc = model.evaluate(test_images,  test_labels, verbose=2)

In [ ]:
model.save('my_model.h5')

In [ ]:
"""
Run this cell, by replacing "<IMAGE_PATH_HERE>" with your image path to test a custom image
"""
# Recreate the exact same model, including its weights and the optimizer
new_model = tf.keras.models.load_model('my_model.h5')

def test(img_path):
    img = cv2.imread(img_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (32, 32))
    img = img / 255.0
    res = model.predict(np.array([img]))
    ans = np.argmax(res)
    print(class_names[ans])

test("<IMAGE_PATH_HERE>")